# Eff. matching to GEN particle


   * [Efficiency vs $\eta$](#Effinciency-vs-$\eta$)
   * [Efficiency vs $p_{T}$](#Efficiency-vs-$p_{T}$)
   * [Turn-on curves](#Turn-on-curves)


In [2]:
import sys
sys.path.insert(0, '..')
import ROOT
import root_numpy as rnp
import l1THistos as histos
import math
import uuid
import pandas as pd

ROOT.enableJSVis()
#ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()
    

In [3]:
# %load ../drawingTools
import ROOT
import math
import uuid

# some useful globals, mainly to deal with ROOT idiosyncrasies
c_idx = 0
p_idx = 0
colors = range(1, 6)
stuff = []
f_idx = 0

ROOT.gStyle.SetPadBottomMargin(0.13)
ROOT.gStyle.SetPadLeftMargin(0.13)
ROOT.gStyle.SetPadRightMargin(0.30)

ROOT.gStyle.SetCanvasBorderMode(0)
ROOT.gStyle.SetCanvasColor(0)
ROOT.gStyle.SetCanvasDefH(600)
ROOT.gStyle.SetCanvasDefW(800)

# define some utility functions
def newCanvas(name=None, title=None, xdiv=0, ydiv=0, form=4):
    global c_idx
    if name is None:
        name = 'c_{}'.format(uuid.uuid4().hex[:6])
        c_idx += 1
    if title is None:
        title = name
    # print name, title
    canvas = ROOT.TCanvas(name, title)
    if(xdiv*ydiv != 0):
        canvas.Divide(xdiv, ydiv)
    global stuff
    stuff.append(canvas)
    return canvas


def draw(plot, options='', text=None):
    c = newCanvas()
    c.cd()
    plot.Draw(options)
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')

    c.Draw()

    return


def getLegend(x1=0.7, y1=0.71, x2=0.95, y2=0.85):
    global stuff
    legend = ROOT.TLegend(x1, y1, x2, y2)
    stuff.append(legend)
    legend.SetFillColor(0)
    legend.SetFillStyle(0)
    legend.SetBorderSize(0)
    legend.SetTextSize(0.05)
    return legend


def drawAndProfileX(plot2d, miny=None, maxy=None, do_profile=True, options='', text=None):
    global p_idx
    if miny and maxy:
        plot2d.GetYaxis().SetRangeUser(miny, maxy)
    c = newCanvas()
    c.SetGrid(1, 1)
    c.cd()
    plot2d.Draw(options)
    ROOT.gPad.SetGrid(1, 1)
    ROOT.gStyle.SetGridColor(15)

    if do_profile:
        profname = plot2d.GetName()+'_prof_'+str(p_idx)
        p_idx += 1
        firstbin = 1
        lastbin = -1
        prof = plot2d.ProfileX(profname, firstbin, lastbin, 's')
        prof.SetMarkerColor(2)
        prof.SetLineColor(2)
        prof.Draw('same')

    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')

    c.Draw()


def getText(text, ndc_x, ndc_y):
    global stuff
    rtext = ROOT.TLatex(ndc_x, ndc_y, text)
    stuff.append(rtext)
    rtext.SetNDC(True)
    # rtext.SetTextFont(40)
    rtext.SetTextSize(0.03)
    return rtext


def drawSame(histograms,
             labels,
             options='',
             norm=False,
             logy=False,
             min_y=None,
             max_y=None,
             text=None,
             y_axis_label=None,
             x_axis_label=None,
             v_lines=None,
             h_lines=None):
    global colors
    global stuff
    c = newCanvas(title=histograms[0].GetName())
    c.cd()
    leg = getLegend()

    max_value = max_y
    min_value = min_y
    if min_y is None:
        min_value = min([hist.GetBinContent(hist.GetMinimumBin()) for hist in histograms])
    if max_y is None:
        max_value = max([hist.GetBinContent(hist.GetMaximumBin()) for hist in histograms])*1.2

    for hidx, hist in enumerate(histograms):
        hist.SetLineColor(colors[hidx])
        if 'TH' in hist.ClassName():
            hist.SetStats(False)
        if norm:
            hist.DrawNormalized('same'+','+options, 1.)
        else:
            if hidx:
                hist.Draw('same'+','+options)
            else:
                hist.Draw(options)
        leg.AddEntry(histograms[hidx], labels[hidx], 'l')

    histograms[0].GetYaxis().SetRangeUser(min_value, max_value)
    if y_axis_label:
        histograms[0].GetYaxis().SetTitle(y_axis_label)
    if x_axis_label:
            histograms[0].GetXaxis().SetTitle(x_axis_label)

    leg.Draw()
    c.Draw()
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")
    if logy:
        c.SetLogy()

    if v_lines:
        for v_line_x in v_lines:
            aline = ROOT.TLine(v_line_x, c.GetUymin(), v_line_x,  c.GetUymax())
            aline.SetLineStyle(2)
            aline.Draw("same")
            stuff.append(aline)
    if h_lines:
        for h_line_y in h_lines:
            aline = ROOT.TLine(c.GetUxmin(), h_line_y, c.GetUxmax(),  h_line_y)
            aline.SetLineStyle(2)
            aline.Draw("same")
            stuff.append(aline)
    c.Update()


def drawProfileX(histograms, labels, options=''):
    profiles = [hist.ProfileX() for hist in histograms]
    drawSame(profiles, labels, options)


def drawSeveral(histograms, labels, options='', do_profile=False, miny=None, maxy=None, text=None):
    ydiv = int(math.ceil(float(len(histograms))/2))
    for hidx in range(0, len(histograms)):
        newtext = labels[hidx]
        if text:
            newtext = '{}: {}'.format(labels[hidx], text)
        if do_profile:
            drawAndProfileX(histograms[hidx], miny=miny, maxy=maxy, options=options, do_profile=do_profile, text=newtext)
        else:
            draw(histograms[hidx], options=options, text=newtext)


def drawProfileRatio(prof1, prof2, ymin=None, ymax=None, text=None):
    hist1 = prof1.ProjectionX(uuid.uuid4().hex[:6])
    hist2 = prof2.ProjectionX(uuid.uuid4().hex[:6])
    hist1.Divide(hist2)
    draw(hist1)
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")

    if ymin is not None and ymax is not None:
        hist1.GetYaxis().SetRangeUser(ymin, ymax)
    ROOT.gPad.Update()


# mean+-nsigmas*RMS.
def drawGaussFit(histo, nsigmas, min, max):
    minfit = histo.GetMean() - nsigmas*histo.GetRMS()
    maxfit = histo.GetMean() + nsigmas*histo.GetRMS()
    drawGFit(histo, min, max, minfit, maxfit)


# Fit a histogram in the range (minfit, maxfit) with a gaussian and
# draw it in the range (min, max)
def drawGFit(histo, min, max, minfit, maxfit):
    # static int i = 0
    # i++
    # gPad->SetGrid(1,1);
    # gStyle->SetGridColor(15);
    histo.GetXaxis().SetRangeUser(min,max)
    global f_idx
    nameF1 = "g{}".format(f_idx)
    f_idx +=1
    g1 = ROOT.TF1(nameF1,"gaus",minfit,maxfit)
    g1.SetLineColor(2)
    g1.SetLineWidth(2)
    histo.Fit(g1,"R")


def drawGraphsSame(histograms, labels, options='', norm=False, logy=False, min_y=None, max_y=None, text=None):
    global colors
    c = newCanvas()
    c.cd()
    leg = getLegend()

    for hidx in range(0, len(histograms)):
        histograms[hidx].SetLineColor(colors[hidx])
        histograms[hidx].Draw('same'+','+options)
        leg.AddEntry(histograms[hidx], labels[hidx], 'l')

    max_value = max_y
    min_value = min_y
    if min_y is None:
        min_value = min([hist.GetBinContent(hist.GetMinimumBin()) for hist in histograms])
    if max_y is None:
        max_value = max([hist.GetBinContent(hist.GetMaximumBin()) for hist in histograms])*1.2
    histograms[0].GetYaxis().SetRangeUser(min_value, max_value)
    leg.Draw()
    c.Draw()
    if logy:
        c.SetLogy()
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")
    c.Update()


In [34]:
# %load samples.py
import ROOT

version = 'v48'


class Sample():
    def __init__(cls, name, label, version=None):
        cls.name = name
        cls.label = label
        if version:
            version = '_'+version
        else:
            version = ''
        cls.histo_filename = '../plots1/histos_{}{}.root'.format(cls.name, version)
        cls.histo_file = ROOT.TFile(cls.histo_filename)


sample_names = ['ele_flat2to100_PU0',
                'ele_flat2to100_PU200',
                'photonPt35_PU0',
                'photonPt35_PU200']

sample_ele_flat2to100_PU0 = Sample('ele_flat2to100_PU0', 'PU0', version)
sample_ele_flat2to100_PU200 = Sample('ele_flat2to100_PU200', 'PU200', version)

sample_gPt35_PU0 = Sample('photonPt35_PU0', 'Pt35 PU0', version)
sample_gPt35_PU200 = Sample('photonPt35_PU200', 'Pt35 PU200', version)

sample_hadronGun_PU0 = Sample('hadronGun_PU0', 'PU0', version)
sample_hadronGun_PU200 = Sample('hadronGun_PU200', 'PU200', version)

samples_ele = [sample_ele_flat2to100_PU0, sample_ele_flat2to100_PU200]

samples_photon = [sample_gPt35_PU0, sample_gPt35_PU200]
samples_hadrons = [sample_hadronGun_PU0, sample_hadronGun_PU200]

sample_nugunrate = Sample('nugun_alleta_pu200', 'PU200', version)
samples_nugunrates = [sample_nugunrate]


tpset_labels = {'DEF': 'NNDR',
                'DEF_em': 'NNDR + EGID',
                'DEF_em_calib': 'NNDR + EGID + calib v1',
                'DEF_emL': 'NNDR + EGIDv1',
                'DEF_emL_calib': 'NNDR + EGIDv1 + calib v1',
                'DEF_pt10': 'NNDR, p_{T}^{L1}>10GeV',
                'DEF_pt20': 'NNDR, p_{T}^{L1}>20GeV',
                'DEF_pt25': 'NNDR, p_{T}^{L1}>25GeV',
                'DEF_pt30': 'NNDR, p_{T}^{L1}>30GeV',
                'DEF_pt10_em': 'NNDR + EGID, p_{T}^{L1}>10GeV',
                'DEF_pt20_em': 'NNDR + EGID, p_{T}^{L1}>20GeV',
                'DEF_pt25_em': 'NNDR + EGID, p_{T}^{L1}>25GeV',
                'DEF_pt30_em': 'NNDR + EGID, p_{T}^{L1}>30GeV',
                'DEF_pt10_emL': 'NNDR + EGIDv1, p_{T}^{L1}>10GeV',
                'DEF_pt20_emL': 'NNDR + EGIDv1, p_{T}^{L1}>20GeV',
                'DEF_pt25_emL': 'NNDR + EGIDv1, p_{T}^{L1}>25GeV',
                'DEF_pt30_emL': 'NNDR + EGIDv1, p_{T}^{L1}>30GeV',
                'DEF_pt10_em_calib': 'NNDR + EGID + calib, p_{T}^{L1}>10GeV',
                'DEF_pt20_em_calib': 'NNDR + EGID + calib, p_{T}^{L1}>20GeV',
                'DEF_pt25_em_calib': 'NNDR + EGID + calib, p_{T}^{L1}>25GeV',
                'DEF_pt30_em_calib': 'NNDR + EGID + calib, p_{T}^{L1}>30GeV',
                'DEF_pt10_emL_calib':  'NNDR + EGIDv1 + calib, p_{T}^{L1}>10GeV',
                'DEF_pt20_emL_calib': 'NNDR + EGIDv1 + calib, p_{T}^{L1}>20GeV',
                'DEF_pt25_emL_calib': 'NNDR + EGIDv1 + calib, p_{T}^{L1}>25GeV',
                'DEF_pt30_emL_calib': 'NNDR + EGIDv1 + calib, p_{T}^{L1}>30GeV'
                }


particle_labels = {'ele': 'all #eta',
                   'elePt20': 'p_{T}^{GEN}>20GeV',
                   'elePt30': 'p_{T}^{GEN}>30GeV',
                   'elePt40': 'p_{T}^{GEN}>40GeV',
                   'eleA': '|#eta^{GEN}| <= 1.52',
                   'eleB': '1.52 < |#eta^{GEN}| <= 1.7',
                   'eleC': '1.7 < |#eta^{GEN}| <= 2.4',
                   'eleD': '2.4 < |#eta^{GEN}| <= 2.8',
                   'eleE': '|#eta^{GEN}| > 2.8',
                   'eleAB': '|#eta^{GEN}| <= 1.7',
                   'eleABC': '|#eta^{GEN}| <= 2.4',
                   'eleBC': '1.52 < |#eta^{GEN}| <= 2.4',
                   'eleBCD': '1.52 < |#eta^{GEN}| <= 2.8',
                   'eleBCDE': '|#eta^{GEN}| > 1.52',
                   'all': 'all #eta^{L1}',
                   'etaA': '|#eta^{L1}| <= 1.52',
                   'etaB': '1.52 < |#eta^{L1}| <= 1.7)',
                   'etaC': '1.7 < |#eta^{L1}| <= 2.4)',
                   'etaD': '2.4 < |#eta^{L1}| <= 2.8)',
                   'etaE': '|#eta^{L1}| > 2.8',
                   'etaAB': '|#eta^{L1}| <= 1.7',
                   'etaABC': '|#eta^{L1}| <= 2.4',
                   'etaBC': '1.52 < |#eta^{L1}| <= 2.4',
                   'etaBCD': '1.52 < |#eta^{L1}| <= 2.8',
                   'etaBCDE': '|#eta^{L1}| > 1.52'}

samples = []
# particles = ''


Error in <TFile::TFile>: file ../plots1/histos_photonPt35_PU0_v48.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photonPt35_PU200_v48.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_hadronGun_PU0_v48.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_hadronGun_PU200_v48.root does not exist


In [35]:
tp_set = 'DEF_pt20_emL'
label = tpset_labels[tp_set]


In [36]:
%%time

samples = samples_ele
tp_sets = ['DEF', 
           'DEF_em', 
           'DEF_em_calib', 
           'DEF_emL', 
           'DEF_emL_calib', 
           'DEF_pt10', 
           'DEF_pt20', 
           'DEF_pt25', 
           'DEF_pt30', 
           'DEF_pt10_em', 
           'DEF_pt20_em', 
           'DEF_pt25_em', 
           'DEF_pt30_em',
           'DEF_pt10_em_calib', 
           'DEF_pt20_em_calib', 
           'DEF_pt25_em_calib', 
           'DEF_pt30_em_calib',
           'DEF_pt10_emL', 
           'DEF_pt20_emL', 
           'DEF_pt25_emL', 
           'DEF_pt30_emL',
           'DEF_pt10_emL_calib', 
           'DEF_pt20_emL_calib', 
           'DEF_pt25_emL_calib', 
           'DEF_pt30_emL_calib'
          ]
# particles = ['ele', 'eleA']

histo_all_df = pd.DataFrame(columns=['histo', 'name', 'sample_name', 'pu', 'tp_set', 'particle', 'text'])


for sample in samples:
    for tp_set in tp_sets:
        for particle in particle_labels.keys():
        # for particle in particles:
            print 'indexing sample: {}, tp_set: {}, particle: {}'.format(sample.name, tp_set, particle)
            histoSet_eff = histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file)
            label = tpset_labels[tp_set]
            histo_all_df = histo_all_df.append({'name' : 'h_eff',
                                                'histo' : histoSet_eff.h_eff,
                                                'sample_name': sample.name,
                                                'pu' : sample.label,
                                                'tp_set': tp_set,
                                                'particle' : particle,
                                                'text' : '{}, {}'.format(tpset_labels[tp_set], particle_labels[particle])
                                                }, ignore_index=True)
            histo_all_df = histo_all_df.append({'name' : 'h_num',
                                                'histo' : histoSet_eff.h_num,
                                                'sample_name': sample.name,
                                                'pu' : sample.label,
                                                'tp_set': tp_set,
                                                'particle' : particle,
                                                'text' : '{}, {}'.format(tpset_labels[tp_set], particle_labels[particle])
                                                }, ignore_index=True)



            
   

indexing sample: ele_flat2to100_PU0, tp_set: DEF, particle: all
indexing sample: ele_flat2to100_PU0, tp_set: DEF, particle: etaABC
indexing sample: ele_flat2to100_PU0, tp_set: DEF, particle: etaD
indexing sample: ele_flat2to100_PU0, tp_set: DEF, particle: etaE
indexing sample: ele_flat2to100_PU0, tp_set: DEF, particle: etaBCDE
indexing sample: ele_flat2to100_PU0, tp_set: DEF, particle: eleBC
indexing sample: ele_flat2to100_PU0, tp_set: DEF, particle: etaA
indexing sample: ele_flat2to100_PU0, tp_set: DEF, particle: etaB
indexing sample: ele_flat2to100_PU0, tp_set: DEF, particle: etaC
indexing sample: ele_flat2to100_PU0, tp_set: DEF, particle: eleBCDE
indexing sample: ele_flat2to100_PU0, tp_set: DEF, particle: ele
indexing sample: ele_flat2to100_PU0, tp_set: DEF, particle: eleBCD
indexing sample: ele_flat2to100_PU0, tp_set: DEF, particle: elePt40
indexing sample: ele_flat2to100_PU0, tp_set: DEF, particle: etaAB
indexing sample: ele_flat2to100_PU0, tp_set: DEF, particle: eleA
indexing sam

indexing sample: ele_flat2to100_PU0, tp_set: DEF_emL_calib, particle: eleC
indexing sample: ele_flat2to100_PU0, tp_set: DEF_emL_calib, particle: eleD
indexing sample: ele_flat2to100_PU0, tp_set: DEF_emL_calib, particle: eleE
indexing sample: ele_flat2to100_PU0, tp_set: DEF_emL_calib, particle: eleAB
indexing sample: ele_flat2to100_PU0, tp_set: DEF_emL_calib, particle: elePt20
indexing sample: ele_flat2to100_PU0, tp_set: DEF_emL_calib, particle: etaBCD
indexing sample: ele_flat2to100_PU0, tp_set: DEF_emL_calib, particle: eleABC
indexing sample: ele_flat2to100_PU0, tp_set: DEF_emL_calib, particle: etaBC
indexing sample: ele_flat2to100_PU0, tp_set: DEF_emL_calib, particle: elePt30
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10, particle: all
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10, particle: etaABC
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10, particle: etaD
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10, particle: etaE
indexing sample: ele_flat2to100_

indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10_em, particle: etaA
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10_em, particle: etaB
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10_em, particle: etaC
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10_em, particle: eleBCDE
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10_em, particle: ele
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10_em, particle: eleBCD
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10_em, particle: elePt40
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10_em, particle: etaAB
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10_em, particle: eleA
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10_em, particle: eleB
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10_em, particle: eleC
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10_em, particle: eleD
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10_em, particle: eleE
indexing sample: ele_flat2to100_PU0, tp_set

indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10_em_calib, particle: eleC
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10_em_calib, particle: eleD
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10_em_calib, particle: eleE
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10_em_calib, particle: eleAB
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10_em_calib, particle: elePt20
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10_em_calib, particle: etaBCD
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10_em_calib, particle: eleABC
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10_em_calib, particle: etaBC
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10_em_calib, particle: elePt30
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt20_em_calib, particle: all
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt20_em_calib, particle: etaABC
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt20_em_calib, particle: etaD
indexing sample: ele_flat2to100_PU0, tp

indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10_emL, particle: elePt20
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10_emL, particle: etaBCD
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10_emL, particle: eleABC
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10_emL, particle: etaBC
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt10_emL, particle: elePt30
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt20_emL, particle: all
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt20_emL, particle: etaABC
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt20_emL, particle: etaD
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt20_emL, particle: etaE
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt20_emL, particle: etaBCDE
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt20_emL, particle: eleBC
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt20_emL, particle: etaA
indexing sample: ele_flat2to100_PU0, tp_set: DEF_pt20_emL, particle: etaB
indexing sample: ele_f

...

indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt10, particle: elePt20
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt10, particle: etaBCD
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt10, particle: eleABC
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt10, particle: etaBC
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt10, particle: elePt30
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20, particle: all
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20, particle: etaABC
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20, particle: etaD
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20, particle: etaE
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20, particle: etaBCDE
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20, particle: eleBC
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20, particle: etaA
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20, particle: etaB
indexing sample: ele_flat2to100_PU200, tp_set: D

indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_em, particle: etaA
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_em, particle: etaB
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_em, particle: etaC
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_em, particle: eleBCDE
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_em, particle: ele
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_em, particle: eleBCD
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_em, particle: elePt40
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_em, particle: etaAB
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_em, particle: eleA
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_em, particle: eleB
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_em, particle: eleC
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_em, particle: eleD
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_em, particle: eleE
indexing sample: 

indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_em_calib, particle: eleBCD
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_em_calib, particle: elePt40
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_em_calib, particle: etaAB
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_em_calib, particle: eleA
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_em_calib, particle: eleB
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_em_calib, particle: eleC
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_em_calib, particle: eleD
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_em_calib, particle: eleE
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_em_calib, particle: eleAB
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_em_calib, particle: elePt20
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_em_calib, particle: etaBCD
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_em_calib, particle: eleABC
indexing sampl

indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_emL, particle: eleA
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_emL, particle: eleB
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_emL, particle: eleC
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_emL, particle: eleD
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_emL, particle: eleE
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_emL, particle: eleAB
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_emL, particle: elePt20
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_emL, particle: etaBCD
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_emL, particle: eleABC
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_emL, particle: etaBC
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_emL, particle: elePt30
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt25_emL, particle: all
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt25_emL, particle: etaABC

indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_emL_calib, particle: eleE
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_emL_calib, particle: eleAB
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_emL_calib, particle: elePt20
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_emL_calib, particle: etaBCD
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_emL_calib, particle: eleABC
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_emL_calib, particle: etaBC
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt20_emL_calib, particle: elePt30
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt25_emL_calib, particle: all
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt25_emL_calib, particle: etaABC
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt25_emL_calib, particle: etaD
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt25_emL_calib, particle: etaE
indexing sample: ele_flat2to100_PU200, tp_set: DEF_pt25_emL_calib, particle: etaBCDE


In [37]:
for sample in samples:
    for tp_set in ['DEF_pt10','DEF_pt20', 'DEF_pt25', 'DEF_pt30', 
                   'DEF_pt10_em', 'DEF_pt20_em', 'DEF_pt25_em', 'DEF_pt30_em',
                   'DEF_pt10_em_calib', 'DEF_pt20_em_calib', 'DEF_pt25_em_calib', 'DEF_pt30_em_calib']:
        TP_set_den = 'DEF'
        if 'em_calib' in tp_set:
            TP_set_den = 'DEF_em_calib'
        elif 'em' in tp_set:
            TP_set_den = 'DEF_em'
        for particle in particle_labels.keys():
            print 'TP set: {} den: {}'.format(tp_set, TP_set_den)
            histo_den = histo_all_df[(histo_all_df.name == 'h_num') & 
                                     (histo_all_df.sample_name == sample.name) & 
                                     (histo_all_df.particle == particle) & 
                                     (histo_all_df.tp_set == TP_set_den)].histo
            histo_num = histo_all_df[(histo_all_df.name == 'h_num') & 
                                     (histo_all_df.sample_name == sample.name) & 
                                     (histo_all_df.particle == particle) & 
                                     (histo_all_df.tp_set == tp_set)].histo
            histo_all_df = histo_all_df.append({'name' : 'h_turnon',
                                                'histo' : histos.HistoEff(histo_num.values[0], histo_den.values[0], False),
                                                'sample_name': sample.name,
                                                'pu' : sample.label,
                                                'tp_set': tp_set,
                                                'particle' : particle,
                                                'text' : '{}, {}'.format(tpset_labels[tp_set], particle_labels[particle])
                                                }, ignore_index=True)


TP set: DEF_pt10 den: DEF
TP set: DEF_pt10 den: DEF
TP set: DEF_pt10 den: DEF
TP set: DEF_pt10 den: DEF
TP set: DEF_pt10 den: DEF
TP set: DEF_pt10 den: DEF
TP set: DEF_pt10 den: DEF
TP set: DEF_pt10 den: DEF
TP set: DEF_pt10 den: DEF
TP set: DEF_pt10 den: DEF
TP set: DEF_pt10 den: DEF
TP set: DEF_pt10 den: DEF
TP set: DEF_pt10 den: DEF
TP set: DEF_pt10 den: DEF
TP set: DEF_pt10 den: DEF
TP set: DEF_pt10 den: DEF
TP set: DEF_pt10 den: DEF
TP set: DEF_pt10 den: DEF
TP set: DEF_pt10 den: DEF
TP set: DEF_pt10 den: DEF
TP set: DEF_pt10 den: DEF
TP set: DEF_pt10 den: DEF
TP set: DEF_pt10 den: DEF
TP set: DEF_pt10 den: DEF
TP set: DEF_pt10 den: DEF
TP set: DEF_pt20 den: DEF
TP set: DEF_pt20 den: DEF
TP set: DEF_pt20 den: DEF
TP set: DEF_pt20 den: DEF
TP set: DEF_pt20 den: DEF
TP set: DEF_pt20 den: DEF
TP set: DEF_pt20 den: DEF
TP set: DEF_pt20 den: DEF
TP set: DEF_pt20 den: DEF
TP set: DEF_pt20 den: DEF
TP set: DEF_pt20 den: DEF
TP set: DEF_pt20 den: DEF
TP set: DEF_pt20 den: DEF
TP set: DEF_

TP set: DEF_pt25_em_calib den: DEF_em_calib
TP set: DEF_pt25_em_calib den: DEF_em_calib
TP set: DEF_pt25_em_calib den: DEF_em_calib
TP set: DEF_pt25_em_calib den: DEF_em_calib
TP set: DEF_pt25_em_calib den: DEF_em_calib
TP set: DEF_pt25_em_calib den: DEF_em_calib
TP set: DEF_pt25_em_calib den: DEF_em_calib
TP set: DEF_pt25_em_calib den: DEF_em_calib
TP set: DEF_pt25_em_calib den: DEF_em_calib
TP set: DEF_pt25_em_calib den: DEF_em_calib
TP set: DEF_pt25_em_calib den: DEF_em_calib
TP set: DEF_pt25_em_calib den: DEF_em_calib
TP set: DEF_pt25_em_calib den: DEF_em_calib
TP set: DEF_pt25_em_calib den: DEF_em_calib
TP set: DEF_pt25_em_calib den: DEF_em_calib
TP set: DEF_pt25_em_calib den: DEF_em_calib
TP set: DEF_pt25_em_calib den: DEF_em_calib
TP set: DEF_pt25_em_calib den: DEF_em_calib
TP set: DEF_pt25_em_calib den: DEF_em_calib
TP set: DEF_pt25_em_calib den: DEF_em_calib
TP set: DEF_pt30_em_calib den: DEF_em_calib
TP set: DEF_pt30_em_calib den: DEF_em_calib
TP set: DEF_pt30_em_calib den: D

TP set: DEF_pt10_em_calib den: DEF_em_calib
TP set: DEF_pt10_em_calib den: DEF_em_calib
TP set: DEF_pt10_em_calib den: DEF_em_calib
TP set: DEF_pt10_em_calib den: DEF_em_calib
TP set: DEF_pt10_em_calib den: DEF_em_calib
TP set: DEF_pt10_em_calib den: DEF_em_calib
TP set: DEF_pt10_em_calib den: DEF_em_calib
TP set: DEF_pt10_em_calib den: DEF_em_calib
TP set: DEF_pt10_em_calib den: DEF_em_calib
TP set: DEF_pt10_em_calib den: DEF_em_calib
TP set: DEF_pt10_em_calib den: DEF_em_calib
TP set: DEF_pt10_em_calib den: DEF_em_calib
TP set: DEF_pt10_em_calib den: DEF_em_calib
TP set: DEF_pt10_em_calib den: DEF_em_calib
TP set: DEF_pt20_em_calib den: DEF_em_calib
TP set: DEF_pt20_em_calib den: DEF_em_calib
TP set: DEF_pt20_em_calib den: DEF_em_calib
TP set: DEF_pt20_em_calib den: DEF_em_calib
TP set: DEF_pt20_em_calib den: DEF_em_calib
TP set: DEF_pt20_em_calib den: DEF_em_calib
TP set: DEF_pt20_em_calib den: DEF_em_calib
TP set: DEF_pt20_em_calib den: DEF_em_calib
TP set: DEF_pt20_em_calib den: D

In [38]:
histo_all_df

,histo,name,sample_name,pu,tp_set,particle,text
0,<l1THistos.HistoEff instance at 0x113fe5a28>,h_eff,ele_flat2to100_PU0,PU0,DEF,all,"NNDR, all #eta^{L1}"
1,<GenParticleHistos h_effNum_DEF_all>,h_num,ele_flat2to100_PU0,PU0,DEF,all,"NNDR, all #eta^{L1}"
2,<l1THistos.HistoEff instance at 0x113e46ea8>,h_eff,ele_flat2to100_PU0,PU0,DEF,etaABC,"NNDR, |#eta^{L1}| <= 2.4"
3,<GenParticleHistos h_effNum_DEF_etaABC>,h_num,ele_flat2to100_PU0,PU0,DEF,etaABC,"NNDR, |#eta^{L1}| <= 2.4"
4,<l1THistos.HistoEff instance at 0x113d4ee18>,h_eff,ele_flat2to100_PU0,PU0,DEF,etaD,"NNDR, 2.4 < |#eta^{L1}| <= 2.8)"
5,<GenParticleHistos h_effNum_DEF_etaD>,h_num,ele_flat2to100_PU0,PU0,DEF,etaD,"NNDR, 2.4 < |#eta^{L1}| <= 2.8)"
6,<l1THistos.HistoEff instance at 0x113fe52d8>,h_eff,ele_flat2to100_PU0,PU0,DEF,etaE,"NNDR, |#eta^{L1}| > 2.8"
7,<GenParticleHistos h_effNum_DEF_etaE>,h_num,ele_flat2to100_PU0,PU0,DEF,etaE,"NNDR, |#eta^{L1}| > 2.8"
8,<l1THistos.HistoEff instance at 0x10ebb5680>,h_eff,ele_flat2to100_PU0,PU0,DEF,etaBCDE,"NNDR, |#eta^{L1}| > 1.52"
9,<GenParticleHistos h_effNum_DEF_etaBCDE>,h_num,ele_flat2to100_PU0,PU0,DEF,etaBCDE,"NNDR, |#eta^{L1}| > 1.52"


## Effinciency vs $\eta$

In [46]:
line_m1p7 = ROOT.TLine(-1.7,0, -1.7, 1.1)
line_m1p7.SetLineStyle(2)
line_p1p7 = ROOT.TLine(1.7,0, 1.7, 1.1)
line_p1p7.SetLineStyle(2)
line_m2p4 = ROOT.TLine(-2.4,0, -2.4, 1.1)
line_m2p4.SetLineStyle(2)
line_p2p4 = ROOT.TLine(2.4,0, 2.4, 1.1)
line_p2p4.SetLineStyle(2)

line_m2p8 = ROOT.TLine(-2.8,0, -2.8, 1.1)
line_m2p8.SetLineStyle(2)
line_p2p8 = ROOT.TLine(2.8,0, 2.8, 1.1)
line_p2p8.SetLineStyle(2)

In [47]:

for particle in ['ele', 'elePt20', 'elePt30', 'elePt40']:
    for tp_set in ['DEF', 'DEF_em', 'DEF_em_calib', 'DEF_emL', 'DEF_emL_calib']:
        histos_sel = histo_all_df[(histo_all_df.tp_set == tp_set) & 
                              (histo_all_df.particle == particle) &
                              (histo_all_df.name == 'h_eff')]
        h_eff_vs_eta = [histo.h_eta.CreateGraph() for histo in histos_sel.histo]
        drawGraphsSame(h_eff_vs_eta, labels=histos_sel.pu.values, text=histos_sel.text.iloc[0], min_y=0, max_y=1.1)
        line_m1p7.Draw('same')
        line_p1p7.Draw('same')
        line_m2p4.Draw('same')
        line_p2p4.Draw('same')
        line_m2p8.Draw('same')
        line_p2p8.Draw('same')

here
there
here
there
here
there
here
there
here
there
here
there
here
there
here
there
here
there
here
there
here
there
here
there
here
there
here
there
here
there
here
there
here
there
here
there
here
there
here
there


In [41]:

for particle in ['elePt30']:
    for tp_set in ['DEF_pt20_em', 'DEF_pt20_em_calib']:
        histos_sel = histo_all_df[(histo_all_df.tp_set == tp_set) & 
                              (histo_all_df.particle == particle) &
                              (histo_all_df.name == 'h_eff')]
        h_eff_vs_eta = [histo.h_eta.CreateGraph() for histo in histos_sel.histo]
        drawGraphsSame(h_eff_vs_eta, labels=histos_sel.pu.values, text=histos_sel.text.iloc[0], min_y=0, max_y=1.1)
        line_m1p7.Draw('same')
        line_p1p7.Draw('same')
        line_m2p8.Draw('same')
        line_p2p8.Draw('same')

here
there
here
there


In [42]:
for particle in ['elePt30']:
    for pu in ['PU0', 'PU200']:
        histos_sel = histo_all_df[(histo_all_df.tp_set.isin(['DEF_pt20_em', 'DEF_pt20_em_calib'])) & 
                              (histo_all_df.particle == particle) &
                              (histo_all_df.name == 'h_eff') &
                              (histo_all_df.pu == pu)]
        h_eff_vs_eta = [histo.h_eta.CreateGraph() for histo in histos_sel.histo]
        drawGraphsSame(h_eff_vs_eta, labels=['nocalib', 'calib'], text=histos_sel.text.iloc[0], min_y=0, max_y=1.1)
        line_m1p7.Draw('same')
        line_p1p7.Draw('same')
        line_m2p8.Draw('same')
        line_p2p8.Draw('same')

here
there
here
there


In [43]:
for particle in ['ele', 'elePt20', 'elePt30', 'elePt40']:
    for pu in ['PU0', 'PU200']:
        histos_sel = histo_all_df[(histo_all_df.tp_set.isin(['DEF_em', 'DEF_em_calib'])) & 
                              (histo_all_df.particle == particle) &
                              (histo_all_df.name == 'h_eff') & 
                              (histo_all_df.pu == pu)]
        h_eff_vs_eta = [histo.h_eta.CreateGraph() for histo in histos_sel.histo]
        drawGraphsSame(h_eff_vs_eta, labels=histos_sel.text.values, text=histos_sel.pu.iloc[0], min_y=0, max_y=1.1)
        line_m1p7.Draw('same')
        line_p1p7.Draw('same')
        line_m2p8.Draw('same')
        line_p2p8.Draw('same')

here
there
here
there
here
there
here
there
here
there
here
there
here
there
here
there


## Efficiency vs $p_{T}$

In [44]:
line_y1p0 = ROOT.TLine(0.,1., 110., 1.)
line_y0p95 = ROOT.TLine(0.,0.95, 110., 0.95)
line_y0p95.SetLineStyle(2)

In [45]:
particles = ['ele', 'eleA', 'eleB', 'eleC', 'eleD', 'eleE', 'eleABC', 'eleBC']

for particle in particles:
    for tp_set in ['DEF', 'DEF_em', 'DEF_em_calib', 'DEF_emL', 'DEF_emL_calib']:
        histos_sel = histo_all_df[(histo_all_df.tp_set == tp_set) & 
                              (histo_all_df.particle == particle) &
                              (histo_all_df.name == 'h_eff')]
        h_eff_vs_pt = [histo.h_pt.CreateGraph() for histo in histos_sel.histo]
#         print h_eff_vs_pt
        drawSame(h_eff_vs_pt, labels=histos_sel.pu.values, text=histos_sel.text.iloc[0], min_y=0, max_y=1.1,y_axis_label='L1 matched to GEN/GEN')
        line_y1p0.Draw('same')
        line_y0p95.Draw('same')
        
        

In [ ]:
particles = ['ele', 'eleA', 'eleB', 'eleC', 'eleD', 'eleE', 'eleABC', 'eleBC']

for particle in particles:
    for pu in ['PU0', 'PU200']:
        histos_sel = histo_all_df[(histo_all_df.tp_set.isin(['DEF_em', 'DEF_em_calib'])) & 
                              (histo_all_df.particle == particle) &
                              (histo_all_df.name == 'h_eff') & 
                              (histo_all_df.pu == pu)]
        h_eff_vs_pt = [histo.h_pt.CreateGraph() for histo in histos_sel.histo]
#         print h_eff_vs_pt
        drawGraphsSame(h_eff_vs_pt, labels=histos_sel.text.values, text=histos_sel.pu.iloc[0], min_y=0, max_y=1.1)
        line_y1p0.Draw('same')
        line_y0p95.Draw('same')
        
        

## Turn-on curves

In [32]:
def doTHDraw(histograms, legend, labels, options):
    for hidx, hist in enumerate(histograms):
        hist.SetLineColor(colors[hidx])
        hist.SetStats(False)
    if norm:
        hist.DrawNormalized('same'+','+options, 1.)
    else:
        if hidx:
            hist.Draw('same'+','+options)
        else:
            hist.Draw(options)
    legend.AddEntry(histograms[hidx], labels[hidx], 'l')

    
def doGraphDraw(histograms, legend, labels, options):
    for hidx in range(0, len(histograms)):
        histograms[hidx].SetLineColor(colors[hidx])
        histograms[hidx].Draw('same'+options)
        legend.AddEntry(histograms[hidx], labels[hidx], 'l')

    
def drawSame(histograms,
             labels,
             options='',
             norm=False,
             logy=False,
             min_y=None,
             max_y=None,
             text=None,
             y_axis_label=None,
             x_axis_label=None,
             v_lines=None,
             h_lines=None):
    global colors
    global stuff
    c = newCanvas(title=histograms[0].GetName())
    c.cd()
    leg = getLegend()


    if 'TH' in histograms[0].ClassName():
        doTHDraw(histograms, leg, labels, options)
    else:
        doGraphDraw(histograms, leg, labels, options)
    
    # now we set the axis range in a sensible manner
    max_value = max_y
    min_value = min_y
    if min_y is None:
        min_value = min([hist.GetBinContent(hist.GetMinimumBin()) for hist in histograms])
    if max_y is None:
        max_value = max([hist.GetBinContent(hist.GetMaximumBin()) for hist in histograms])*1.2
    histograms[0].GetYaxis().SetRangeUser(min_value, max_value)

    # we edit the axis labels
    if y_axis_label:
        histograms[0].GetYaxis().SetTitle(y_axis_label)
    if x_axis_label:
            histograms[0].GetXaxis().SetTitle(x_axis_label)

    leg.Draw()
    c.Draw()

    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")

    if logy:
        c.SetLogy()

    if v_lines:
        for v_line_x in v_lines:
            aline = ROOT.TLine(v_line_x, c.GetUymin(), v_line_x,  c.GetUymax())
            aline.SetLineStyle(2)
            aline.Draw("same")
            stuff.append(aline)
            
    if h_lines:
        for h_line_y in h_lines:
            aline = ROOT.TLine(c.GetUxmin(), h_line_y, c.GetUxmax(),  h_line_y)
            aline.SetLineStyle(2)
            aline.Draw("same")
            stuff.append(aline)
    c.Update()

    
def drawGraphsSame(histograms, 
                   labels, 
                   options='', 
                   norm=False, 
                   logy=False, 
                   min_y=None, 
                   max_y=None, 
                    text=None):
    global colors
    c = newCanvas()
    c.cd()
    leg = getLegend()

    for hidx in range(0, len(histograms)):
        histograms[hidx].SetLineColor(colors[hidx])
        if hidx == 0:
            print 'here'
            histograms[hidx].Draw('same'+options)
        else:
            print 'there'
            histograms[hidx].Draw('same'+options)
        leg.AddEntry(histograms[hidx], labels[hidx], 'l')

    max_value = max_y
    min_value = min_y
    if min_y is None:
        min_value = min([hist.GetBinContent(hist.GetMinimumBin()) for hist in histograms])
    if max_y is None:
        max_value = max([hist.GetBinContent(hist.GetMaximumBin()) for hist in histograms])*1.2
    histograms[0].GetYaxis().SetRangeUser(min_value, max_value)
    
    leg.Draw()
    if logy:
        c.SetLogy()
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")
    c.Update()
    c.Draw()

for tp_set in ['DEF_pt20', 'DEF_pt25', 'DEF_pt30', 
               'DEF_pt20_em', 'DEF_pt25_em', 'DEF_pt30_em',
               'DEF_pt20_em_calib', 'DEF_pt25_em_calib', 'DEF_pt30_em_calib']:
    for particle in particles:
        histos_sel = histo_all_df[(histo_all_df.tp_set == tp_set) & 
                                  (histo_all_df.particle == particle) &
                                  (histo_all_df.name == 'h_turnon')]
            
        h_eff_vs_pt = [histo.h_pt.CreateGraph() for histo in histos_sel.histo]
        print tp_set
        print h_eff_vs_pt
        drawSame(h_eff_vs_pt, labels=histos_sel.pu.values, text=histos_sel.text.iloc[0], min_y=0, max_y=1.1, y_axis_label="L1 matched to GEN and p_{T}>th./L1 matched to GEN")
        line_y1p0.Draw('same')
        line_y0p95.Draw('same')
          
        


DEF_pt20
[<ROOT.TGraphAsymmErrors object ("eff_graph") at 0x7fe2d2957060>, <ROOT.TGraphAsymmErrors object ("eff_graph") at 0x7fe2d2957130>]
DEF_pt20
[<ROOT.TGraphAsymmErrors object ("eff_graph") at 0x7fe2d2addd70>, <ROOT.TGraphAsymmErrors object ("eff_graph") at 0x7fe2d20f0e10>]
DEF_pt20
[<ROOT.TGraphAsymmErrors object ("eff_graph") at 0x7fe2d294a790>, <ROOT.TGraphAsymmErrors object ("eff_graph") at 0x7fe2d294af30>]
DEF_pt20
[<ROOT.TGraphAsymmErrors object ("eff_graph") at 0x7fe2d294a480>, <ROOT.TGraphAsymmErrors object ("eff_graph") at 0x7fe2d294a630>]
DEF_pt20
[<ROOT.TGraphAsymmErrors object ("eff_graph") at 0x7fe2d2add670>, <ROOT.TGraphAsymmErrors object ("eff_graph") at 0x7fe2d2ac4e90>]
DEF_pt20
[<ROOT.TGraphAsymmErrors object ("eff_graph") at 0x7fe2d2ae03e0>, <ROOT.TGraphAsymmErrors object ("eff_graph") at 0x7fe2d2ae0590>]
DEF_pt20
[<ROOT.TGraphAsymmErrors object ("eff_graph") at 0x7fe2d294a0d0>, <ROOT.TGraphAsymmErrors object ("eff_graph") at 0x7fe2d2958f20>]
DEF_pt20
[<ROOT.TGra

DEF_pt30_em_calib
[<ROOT.TGraphAsymmErrors object ("eff_graph") at 0x7fe2d22b7a60>, <ROOT.TGraphAsymmErrors object ("eff_graph") at 0x7fe2d22b8a00>]
DEF_pt30_em_calib
[<ROOT.TGraphAsymmErrors object ("eff_graph") at 0x7fe2d2a23400>, <ROOT.TGraphAsymmErrors object ("eff_graph") at 0x7fe2d2a226c0>]
DEF_pt30_em_calib
[<ROOT.TGraphAsymmErrors object ("eff_graph") at 0x7fe2d2a1ed30>, <ROOT.TGraphAsymmErrors object ("eff_graph") at 0x7fe2d2a1f4d0>]
DEF_pt30_em_calib
[<ROOT.TGraphAsymmErrors object ("eff_graph") at 0x7fe2d3d95cc0>, <ROOT.TGraphAsymmErrors object ("eff_graph") at 0x7fe2d3d95d90>]
DEF_pt30_em_calib
[<ROOT.TGraphAsymmErrors object ("eff_graph") at 0x7fe2c8eda860>, <ROOT.TGraphAsymmErrors object ("eff_graph") at 0x7fe2c8ed7110>]
DEF_pt30_em_calib
[<ROOT.TGraphAsymmErrors object ("eff_graph") at 0x7fe2d22bad90>, <ROOT.TGraphAsymmErrors object ("eff_graph") at 0x7fe2d22bba30>]


In [ ]:
for pu in ['PU0', 'PU200']:
    for tp_set in ['DEF_pt20', 'DEF_pt25', 'DEF_pt30', 'DEF_pt20_em', 'DEF_pt25_em', 'DEF_pt30_em']:
        histos_sel = histo_all_df[(histo_all_df.tp_set == tp_set) & 
                                  (histo_all_df.particle.isin(['eleABC', 'eleBC'])) &
                                  (histo_all_df.name == 'h_turnon') &
                                  (histo_all_df.pu == pu)]

        h_eff_vs_pt = [histo.h_pt.CreateGraph() for histo in histos_sel.histo]
        drawGraphsSame(h_eff_vs_pt, labels=histos_sel.text.values, text=histos_sel.pu.iloc[0], min_y=0, max_y=1.1)
        line_y1p0.Draw('same')
        line_y0p95.Draw('same')
          
        


In [ ]:
for pu in ['PU0', 'PU200']:
    for particle in ['eleB', 'eleC', 'eleD', 'eleABC', 'eleBC', 'eleBCD']:
        for tp_set in [['DEF_pt20_em','DEF_pt20_em_calib'], 
                       ['DEF_pt25_em', 'DEF_pt25_em_calib'], 
                       ['DEF_pt30_em', 'DEF_pt30_em_calib']]:
            histos_sel = histo_all_df[(histo_all_df.tp_set.isin(tp_set)) & 
                                      (histo_all_df.particle == particle) &
                                      (histo_all_df.name == 'h_turnon') &
                                      (histo_all_df.pu == pu)]

            h_eff_vs_pt = [histo.h_pt.CreateGraph() for histo in histos_sel.histo]
            text = '{}, {}'.format(histos_sel.pu.iloc[0], histos_sel.text.iloc[0])
            drawGraphsSame(h_eff_vs_pt, labels=['no calib', 'calib'], text=text, min_y=0, max_y=1.1)
            line_y1p0.Draw('same')
            line_y0p95.Draw('same')

            
            

In [ ]:
def f_yc(x, par):
    return (ROOT.Math.normal_cdf(par[0]*(x[0]-par[1]), par[0]*par[2], 0) - ROOT.TMath.Exp(-par[0]*(x[0]-par[1])+par[0]*par[0]*par[2]*par[2]/2)*ROOT.Math.normal_cdf(par[0]*(x[0]-par[1]), par[0]*par[2], par[0]*par[0]*par[2]*par[2])) * (par[3] - par[4]) + par[4]

tf_yc = ROOT.TF1('tf_yc', f_yc, 0, 100, 5)
tf_yc.SetParameters(1, 20, 4, 1.0, 0.0);
tf_yc.SetParNames("#lambda", "#mu", "#sigma", "Plateau", "Baseline");
tf_yc.SetLineColor(6)
tf_yc.FixParameter(4,0)

tf_yc.SetParLimits(3, 0.8, 1)
tf_yc.SetParLimits(0, 0, 1)

c = newCanvas()
tf_yc.Draw()
c.Draw()


In [ ]:

import numpy as np
from array import array

def f_cb(x, par):
    return ROOT.Math.crystalball_function(x[0], par[0], par[1], par[2], par[3])

tf_cb = ROOT.TF1('cb', f_cb, 0, 100, 4)
tf_cb.SetParameters(10., 4., 3, 20.)

def f_cbi(x, par):
    return ROOT.Math.crystalball_cdf(x[0], par[0], par[1], par[2], par[3])
#     tf_cb.SetParameters(par[0], par[1], par[2], par[3])
#     npts = 1000
#     x_ = []
#     w_ = []
   
#     return tf_cb.IntegralFast(npts, np.array(npts), np.array(npts), 0., x[0])
    #return tf_cb.Integral(0., x[0])
    
    
tf_cbi = ROOT.TF1('cbi', f_cbi, 0, 100, 4)
tf_cbi.SetParameters(10., 4., 3, 20.)
tf_cbi.SetLineColor(4)

c = newCanvas()
tf_cbi.Draw()
c.Draw()


    # ROOT::Math::crystalball_function (double x, double alpha, double n, double sigma, double x0=0)

# fitf_cb = ROOT.TF1('cbfunc', 'ROOT.TMath.crystalball_function (x, [0], [1], [2], [3])', 0, 100)
# 

# fitf_cb.SetParameters(1, 1, 1, 0)


In [ ]:
# Double_t turnon_func(Double_t *x, Double_t *par)
# {
#   double halfpoint = par[0];
#   double slope = par[1];
#   double plateau = par[2];

#   //double offset = par[3];
#   //double plateau = 1.0;
#   double offset = 0;

#   double pt = TMath::Max(x[0],0.000001);

#    double arg = 0;
#    //cout << pt <<", "<< halfpoint <<", " << slope <<endl;
#    arg = (pt - halfpoint)/(TMath::Sqrt(pt)*slope);
#    double fitval = offset+0.5*plateau*(1+TMath::Erf(arg));
#    return fitval;
# }




def turnon_func1(x, par):

    return 0.5 * par[0] * (1+ROOT.TMath.Erf((x[0]-par[1])/(par[2]*ROOT.TMath.Sqrt(2))))




def turnon_func(x, par):
  
    halfpoint = par[0]
    slope = par[1]
    plateau = par[2]
    offset = 0
    pt = ROOT.TMath.Max(x[0],0.000001)
    arg = 0
    arg = (pt - halfpoint)/(ROOT.TMath.Sqrt(pt)*slope)
    fitval = offset+0.5*plateau*(1+ROOT.TMath.Erf(arg))
    return fitval

fitf_to = ROOT.TF1('myfunc', turnon_func1, 0, 100, 3)
#fitf.SetParNames('#epsilon', "#mu", '#sigma')
fitf_to.SetParameters(1,25,2)
fitf_to.SetLineColor(3)

def fermi(x, params):
    return params[3]*(params[0] - 1/(ROOT.TMath.Exp((x[0]-params[1])/(params[2]))+1))
  
fitf = ROOT.TF1('myfunc', fermi, 0, 100, 4)
fitf.SetParNames('#epsilon', "#mu", '#sigma')
fitf.SetParameters(1,25,2,1)



c = newCanvas()
fitf.Draw()
c.Draw()


In [ ]:
pu = ''
ROOT.TVirtualFitter.SetDefaultFitter("Fumili2")
ROOT.TVirtualFitter.SetPrecision(1e-04)

points = []



for pu in ['PU0', 'PU200']:
#     for particle in ['eleB', 'eleC', 'eleD', 'eleABC', 'eleBC', 'eleBCD']:
    for particle in ['eleBC']:
        for pt in [10, 20, 25, 30]:
            for tp_set in ['DEF_pt{}_em'.format(pt),'DEF_pt{}_em_calib'.format(pt)]:#, 
    #                        'DEF_pt25_em', 'DEF_pt25_em_calib', 
    #                        'DEF_pt30_em', 'DEF_pt30_em_calib']:
                histos_sel = histo_all_df[(histo_all_df.tp_set == tp_set) & 
                                          (histo_all_df.particle == particle) &
                                          (histo_all_df.name == 'h_turnon') &
                                          (histo_all_df.pu == pu)]            
                h_eff_vs_pt = [histo.h_pt.CreateGraph() for histo in histos_sel.histo]
    #             result = h_eff_vs_pt[0].Fit(fitf, 'QERS+')
    #             text = '{}, {}'.format(histos_sel.pu.iloc[0], histos_sel.text.iloc[0])
    #             print '-------- {}'.format(text)
    #             print result.Print()

    #             result = h_eff_vs_pt[0].Fit(fitf_to, 'QERS+')
    #             text = '{}, {}'.format(histos_sel.pu.iloc[0], histos_sel.text.iloc[0])
    #             print '-------- {}'.format(text)
    #             print result.Print()
                tf_yc.SetParameters(1, 20, 4, 1.0, 0.0);

                result = h_eff_vs_pt[0].Fit(tf_yc, 'MQRS+')
                text = '{}, {}'.format(histos_sel.pu.iloc[0], histos_sel.text.iloc[0])
                print '-------- {}'.format(text)
                print result.Print()
                pt095 = tf_yc.GetX(0.95)
                print 'pt: {}GeV, 0.95 value: {}'.format(pt, pt095)
                points.append((tp_set, pu, pt, pt095))
                drawGraphsSame(h_eff_vs_pt, labels=['no calib', 'calib'], text=text, min_y=0, max_y=1.1)
                ROOT.gStyle.SetOptFit(11111)
                line_y1p0.Draw('same')
                line_y0p95.Draw('same')
          

In [ ]:
points

points_calib_pu0 = [point for point in points if ('calib' in point[0] and 'PU0' in point[1])]
points_calib_pu200 = [point for point in points if ( 'calib' in point[0] and 'PU200' in point[1])]

points_nocalib_pu0 = [point for point in points if ('calib' not in point[0] and 'PU0' in point[1])]
points_nocalib_pu200 = [point for point in points if ( 'calib' not in point[0] and 'PU200' in point[1])]

print points
print points_calib_pu0
print points_calib_pu200
print points_nocalib_pu0
print points_nocalib_pu200

graph_nocalib_pu0 = ROOT.TGraph(len(points_nocalib_pu0), array('d', [point[2] for point in points_nocalib_pu0]), array('d',[point[3] for point in points_nocalib_pu0]))
drawGraphsSame([graph_nocalib_pu0], labels=['PU0'], min_y=0, max_y=50, options='A*', text='no calib')
graph_nocalib_pu0.Fit('pol1', 'S')
ROOT.gStyle.SetOptFit(11111)

graph_calib_pu0 = ROOT.TGraph(len(points_calib_pu0), array('d', [point[2] for point in points_calib_pu0]), array('d',[point[3] for point in points_calib_pu0]))
drawGraphsSame([graph_calib_pu0], labels=['PU0'], min_y=0, max_y=50, options='A*', text='calib')
graph_calib_pu0.Fit('pol1', 'S')
ROOT.gStyle.SetOptFit(11111)


graph_nocalib_pu200 = ROOT.TGraph(len(points_nocalib_pu200), array('d', [point[2] for point in points_nocalib_pu200]), array('d',[point[3] for point in points_nocalib_pu200]))
drawGraphsSame([graph_nocalib_pu200], labels=['PU200'], min_y=0, max_y=50, options='A*', text='no calib')
graph_nocalib_pu200.Fit('pol1', 'S')
ROOT.gStyle.SetOptFit(11111)

graph_calib_pu200 = ROOT.TGraph(len(points_calib_pu200), array('d', [point[2] for point in points_calib_pu200]), array('d',[point[3] for point in points_calib_pu200]))
drawGraphsSame([graph_calib_pu200], labels=['PU200'], min_y=0, max_y=50, options='A*', text='calib')
graph_calib_pu200.Fit('pol1', 'S')
ROOT.gStyle.SetOptFit(11111)

In [ ]:
pu = ''
ROOT.TVirtualFitter.SetDefaultFitter("Fumili2")
ROOT.TVirtualFitter.SetPrecision(1e-04)

points = []



for pu in ['PU0', 'PU200']:
#     for particle in ['eleB', 'eleC', 'eleD', 'eleABC', 'eleBC', 'eleBCD']:
    for particle in ['eleABC']:
        for pt in [10, 20, 25, 30]:
            for tp_set in ['DEF_pt{}_em'.format(pt),'DEF_pt{}_em_calib'.format(pt)]:#, 
    #                        'DEF_pt25_em', 'DEF_pt25_em_calib', 
    #                        'DEF_pt30_em', 'DEF_pt30_em_calib']:
                histos_sel = histo_all_df[(histo_all_df.tp_set == tp_set) & 
                                          (histo_all_df.particle == particle) &
                                          (histo_all_df.name == 'h_turnon') &
                                          (histo_all_df.pu == pu)]            
                h_eff_vs_pt = [histo.h_pt.CreateGraph() for histo in histos_sel.histo]
    #             result = h_eff_vs_pt[0].Fit(fitf, 'QERS+')
    #             text = '{}, {}'.format(histos_sel.pu.iloc[0], histos_sel.text.iloc[0])
    #             print '-------- {}'.format(text)
    #             print result.Print()

    #             result = h_eff_vs_pt[0].Fit(fitf_to, 'QERS+')
    #             text = '{}, {}'.format(histos_sel.pu.iloc[0], histos_sel.text.iloc[0])
    #             print '-------- {}'.format(text)
    #             print result.Print()
                tf_yc.SetParameters(1, 20, 4, 1.0, 0.0);

                result = h_eff_vs_pt[0].Fit(tf_yc, 'MQRS+')
                text = '{}, {}'.format(histos_sel.pu.iloc[0], histos_sel.text.iloc[0])
                print '-------- {}'.format(text)
                print result.Print()
                pt095 = tf_yc.GetX(0.95)
                print 'pt: {}GeV, 0.95 value: {}'.format(pt, pt095)
                points.append((tp_set, pu, pt, pt095))
                drawGraphsSame(h_eff_vs_pt, labels=['no calib', 'calib'], text=text, min_y=0, max_y=1.1)
                ROOT.gStyle.SetOptFit(11111)
                line_y1p0.Draw('same')
                line_y0p95.Draw('same')
          

In [ ]:
points

points_calib_pu0 = [point for point in points if ('calib' in point[0] and 'PU0' in point[1])]
points_calib_pu200 = [point for point in points if ( 'calib' in point[0] and 'PU200' in point[1])]

points_nocalib_pu0 = [point for point in points if ('calib' not in point[0] and 'PU0' in point[1])]
points_nocalib_pu200 = [point for point in points if ( 'calib' not in point[0] and 'PU200' in point[1])]

print points
print points_calib_pu0
print points_calib_pu200
print points_nocalib_pu0
print points_nocalib_pu200

graph_nocalib_pu0 = ROOT.TGraph(len(points_nocalib_pu0), array('d', [point[2] for point in points_nocalib_pu0]), array('d',[point[3] for point in points_nocalib_pu0]))
drawGraphsSame([graph_nocalib_pu0], labels=['PU0'], min_y=0, max_y=50, options='A*', text='no calib')
graph_nocalib_pu0.Fit('pol1', 'S')
ROOT.gStyle.SetOptFit(11111)

graph_calib_pu0 = ROOT.TGraph(len(points_calib_pu0), array('d', [point[2] for point in points_calib_pu0]), array('d',[point[3] for point in points_calib_pu0]))
drawGraphsSame([graph_calib_pu0], labels=['PU0'], min_y=0, max_y=50, options='A*', text='calib')
graph_calib_pu0.Fit('pol1', 'S')
ROOT.gStyle.SetOptFit(11111)


graph_nocalib_pu200 = ROOT.TGraph(len(points_nocalib_pu200), array('d', [point[2] for point in points_nocalib_pu200]), array('d',[point[3] for point in points_nocalib_pu200]))
drawGraphsSame([graph_nocalib_pu200], labels=['PU200'], min_y=0, max_y=50, options='A*', text='no calib')
graph_nocalib_pu200.Fit('pol1', 'S')
ROOT.gStyle.SetOptFit(11111)

graph_calib_pu200 = ROOT.TGraph(len(points_calib_pu200), array('d', [point[2] for point in points_calib_pu200]), array('d',[point[3] for point in points_calib_pu200]))
drawGraphsSame([graph_calib_pu200], labels=['PU200'], min_y=0, max_y=50, options='A*', text='calib')
graph_calib_pu200.Fit('pol1', 'S')
ROOT.gStyle.SetOptFit(11111)

In [ ]:

samples = samples_ele
tp_sets = ['DEF', 'DEF_em', 'DEF_pt20', 'DEF_pt20_em', 'DEF_pt25', 'DEF_pt25_em', 'DEF_pt30', 'DEF_pt30_em']
particles = ['ele', 'eleA', 'eleB', 'eleC', 'eleD', 'eleAAA', 'eleAA', 'eleBB', 'eleCC', 'eleDD']


labels = [sample.label for sample in samples]




histo_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_num_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_den_df = pd.DataFrame(index=tp_sets, columns=particles)
titles_df = pd.DataFrame(index=tp_sets, columns=particles)
# print histo_num_df
for tp_set in tp_sets:
    for particle in particles:
        histo_df.loc[tp_set][particle] =  [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_eff for sample in samples]
        histo_num_df.loc[tp_set][particle] = [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_num for sample in samples]
        histo_den_df.loc[tp_set][particle] = [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_den for sample in samples]

        titles_df.loc[tp_set, particle] = '{}, {}'.format(tpset_labels[tp_set], particle_labels[particle])

histo_df



In [ ]:
titles_df

In [ ]:
dir(histo_df.ele['DEF'][0])


In [ ]:
histo_df.loc['DEF', 'ele']

In [ ]:

for particle in ['ele']:
    for tp_set in tp_sets:
        h_eff_vs_eta = [histo.h_eta.CreateGraph() for histo in histo_df.ele[tp_set]]
        drawGraphsSame(h_eff_vs_eta, labels, text=titles_df.loc[tp_set, particle])



In [ ]:
line095 = ROOT.TLine(0,0.95,110,0.95)
line095.SetLineStyle(2)
for particle in particles:
    for tp_set in tp_sets:
        #print 'particle: {}, TP set: {}, text: {}'.format(particle, tp_set, titles_df.loc[tp_set, particle])
        h_eff_vs_pt = [histo.h_pt.CreateGraph() for histo in histo_df.loc[tp_set, particle]]
        drawGraphsSame(h_eff_vs_pt, labels, text=titles_df.loc[tp_set, particle], min_y=0., max_y=1.1)
        line095.Draw('same')

In [ ]:
histo_den_df

In [ ]:
# build turnon

for particle in particles:
    TP_set_den = 'DEF' # -> L1 matched/GEN
    TP_set_num_l = ['DEF_pt20', 'DEF_pt25', 'DEF_pt30'] # -> L1 matched & above threshols / GEN
    
    histo_den = [histo for histo in histo_num_df.loc[TP_set_den][particle]] # -> L1 matched
    for TP_set_num in TP_set_num_l:
        histo_num = [histo for histo in histo_num_df.loc[TP_set_num][particle]] # -> L1 matched & above threshols

        histos_turnon = []
        for idx,histo_num in enumerate(histo_num):
            histo_turnon_num = histo_num.h_pt.Clone(uuid.uuid4().hex[:6])
            stuff.append(histo_turnon_num)
            histo_turnon_den = histo_den[idx].h_pt.Clone(uuid.uuid4().hex[:6])
            stuff.append(histo_turnon_den)
            hton = ROOT.TEfficiency(histo_turnon_num, histo_turnon_den)
            stuff.append(hton)
            histos_turnon.append(hton)
        drawGraphsSame(histos_turnon, labels, text=titles_df.loc[TP_set_num, particle])

In [ ]:
for particle in particles:
    TP_set_den = 'DEF_em' # -> L1 matched/GEN
    TP_set_num_l = ['DEF_pt20_em', 'DEF_pt25_em', 'DEF_pt30_em'] # -> L1 matched & above threshols / GEN
    
    histo_den = [histo for histo in histo_num_df.loc[TP_set_den][particle]] # -> L1 matched
    for TP_set_num in TP_set_num_l:
        histo_num = [histo for histo in histo_num_df.loc[TP_set_num][particle]] # -> L1 matched & above threshols

        histos_turnon = []
        for idx,histo_num in enumerate(histo_num):
            histo_turnon_num = histo_num.h_pt.Clone(uuid.uuid4().hex[:6])
            stuff.append(histo_turnon_num)
            histo_turnon_den = histo_den[idx].h_pt.Clone(uuid.uuid4().hex[:6])
            stuff.append(histo_turnon_den)
            hton = ROOT.TEfficiency(histo_turnon_num, histo_turnon_den)
            stuff.append(hton)
            histos_turnon.append(hton)
        drawGraphsSame(histos_turnon, labels, text=titles_df.loc[TP_set_num, particle])

# Photon pt35 GeV 1.6 < |eta| < 2.8 

In [ ]:

samples = samples_photon
tp_sets = ['DEF', 'DEF_em']
particles = ['photon']
labels = [sample.label for sample in samples]

histo_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_num_df = pd.DataFrame(index=tp_sets, columns=particles)
histo_den_df = pd.DataFrame(index=tp_sets, columns=particles)

for tp_set in tp_sets:
    for particle in particles:
        histo_df.loc[tp_set][particle] =  [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_eff for sample in samples]
        histo_num_df[tp_set][particle] = [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_num for sample in samples]
        histo_den_df[tp_set][particle] = [histos.HistoSetEff('{}_{}'.format(tp_set, particle), sample.histo_file).h_den for sample in samples]

        histo_df



In [ ]:
dir(histo_df.photon['DEF'][0])


In [ ]:
tp_set = 'DEF'
h_eff_vs_eta = [histo.h_eta.CreateGraph() for histo in histo_df.photon[tp_set]]
drawGraphsSame(h_eff_vs_eta, labels)

tp_set = 'DEF_em'
h_eff_vs_eta = [histo.h_eta.CreateGraph() for histo in histo_df.photon[tp_set]]
drawGraphsSame(h_eff_vs_eta, labels)
